# Getting Latitude & Longitude details

### Below cell is from previous notebook.

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup 

url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
website_as_html = requests.get(url).text
soup = BeautifulSoup(website_as_html,'lxml')
my_table = soup.find('table',{'class':'wikitable sortable'})
tr_tag_data=my_table.findAll('tr')
data={'PostalCode':[],'Borough':[],'Neighborhood':[]}
for i in tr_tag_data[1:]:
    row=i.text.split('\n')[1:-1]
    if row[1]=='Not assigned':
        continue
    elif row[2]=='Not assigned':
        data['Neighborhood'].append(row[1])
    else:
        data['Neighborhood'].append(row[2])
        
    data['PostalCode'].append(row[0])
    data['Borough'].append(row[1])

df=pd.DataFrame(data)
toronto_data=df.sort_values(['PostalCode']).reset_index(drop=True)
toronto_data=toronto_data.groupby(['PostalCode','Borough'], as_index=False).agg(', '.join)
toronto_data.head()

### This cell is to find coordinates using _geocoder_, but it's not working for me

In [ ]:
# import geocoder 

# lat=[]
# lon=[]
# postalcodes=toronto_data['PostalCode'].tolist()

# for code in postalcodes:
#     lat_lng_coords = None
#     
#     while(lat_lng_coords is None):
#       
#         g = geocoder.google('{}, Toronto, Ontario'.format(code))
#         lat_lng_coords = g.latlng

#     lat.append(lat_lng_coords[0])
#     lon.append(lat_lng_coords[1])
#     print(lat_lng_coords[0],lat_lng_coords[1])
    
# for lat,lon in zip (lat,lon):
#     print(lat,lon)

### Therefore, I'm using the provided _.csv_ file

In [ ]:
coodinates=pd.read_csv('Lat_Lng.csv')
coodinates.head()

## Finally merging both the df into single one to get desired df

In [ ]:
toronto_dataset=pd.merge(toronto_data, coodinates, on='PostalCode')
toronto_dataset.head()